In [1]:
import numpy as np
import math as m
import networkx as nx
import matplotlib.pyplot as plt
from time import time
import seaborn as sns

In [2]:
def get_stimulus(A, beta, reward):
    arg = beta*(reward - A)
    return m.tanh(arg)

In [3]:
def update_p(aspiration_level, beta, reward, prev_action, p):
    s = get_stimulus(aspiration_level, beta, reward)
    if prev_action == 0: #previous action = cooperation
        if s >= 0:
            return p + (1-p)*s
        else:
            return p + p*s
    else: #previous action = defect
        if s >= 0:
            return p - p*s
        else:
            return p - (1-p)*s

In [4]:
def misimplement_prob_update(eps, prob):
    return prob*(1-eps) + (1-prob)*eps

In [5]:
def get_payoff_matrix(b,c):
    return np.array([[b-c, -c],
                     [b, 0]])

In [6]:
def get_payoffs(p1, p2):
    return payoff_mat[p1][p2]

In [7]:
def create_ring_graph(N, k):
    return nx.watts_strogatz_graph(N, k, 0)


#pos = nx.circular_layout(G_small_ring)
#plt.figure(3,figsize=(20,20)) 
#nx.draw_networkx(G_small_ring, pos=pos, with_labels=False)


In [8]:
def simulate_game(G, payoff_mat, k, rounds, A, beta, eps):
    nodes = len(list(G.nodes))
    probas = np.array([0.8 for _ in range(nodes)])
    counts = np.zeros(rounds)
    assortment=np.zeros(rounds)
    for r in range(rounds):
        payoffs = np.zeros(nodes)
        actions = np.random.binomial(1, p = (1 - probas))
        #print(actions)
        #assortment[r]=actions.count(0)
        countC=0
        countD=0
        for node in list(G.nodes):
            for neighbour in list(G.neighbors(node)):
                if(actions[neighbour]==0 and actions[node]==0):
                  countC+=1
                elif(actions[neighbour]==0 and actions[node]==1):
                  countD+=1
                payoffs[node] += get_payoffs(actions[node], actions[neighbour])
            payoffs[node]/k
        for node in list(G.nodes):
            probas[node] = misimplement_prob_update(eps, update_p(A, beta, payoffs[node], actions[node], probas[node]))
        counts[r] = nodes - np.count_nonzero(actions)
        assortment[r]=(countC-countD)/(nodes*k)
    assort=np.sum(assortment)/rounds
    print(assort)
    return counts/nodes

In [9]:
N = 100
k = 2
G = create_ring_graph(N, k)
A = 1
beta = 0.2
eps = 0.05
payoff_mat = get_payoff_matrix(6, 1)

In [10]:
coops_lvl = []

#for i in range(1000):
#    coops_lvl.append(simulate_game(G, payoff_mat, k, 25, A, beta, eps))
startTime = time()
A_values = np.linspace(-1, 5, num = 100)
eps_values = np.linspace(0, 0.5, num = 100)

heatmap = np.zeros((100,100))

for a_i in range(len(A_values)):
    for eps_i in range(len(eps_values)):
        heatmap[eps_i][a_i] = simulate_game(G, payoff_mat, k, 25, A_values[a_i], beta, eps_values[eps_i])[-1]

endTime = time()
print("\nSimulating took {} seconds".format(round(endTime - startTime)))
np.save(file = "test.npy", arr = heatmap)
        

0.47239999999999993
0.2696
0.3116
0.5052
0.20800000000000002
0.20800000000000002
0.25880000000000003
0.12
0.10279999999999999
0.0872
0.16399999999999998
0.19079999999999997
0.14959999999999998
0.1176
0.11760000000000001
0.1496
0.11280000000000001
0.11679999999999997
0.09759999999999999
0.1044
0.10679999999999999
0.0492
0.048
0.08039999999999999
0.05279999999999999
0.0276
0.0384
0.0512
0.036
0.062
0.018000000000000002
0.016000000000000004
-0.011200000000000002
-0.009200000000000003
0.028399999999999998
0.08839999999999999
0.024399999999999998
0.046000000000000006
0.039599999999999996
-0.009600000000000004
0.0188
0.0008
0.0168
0.008
0.0352
-0.020400000000000005
-0.008
-0.015600000000000001
0.04
0.036000000000000004
0.0007999999999999985
0.006799999999999996
-0.011200000000000002
-0.023600000000000003
0.0144
0.009199999999999998
0.0368
0.008399999999999998
-0.0044
0.0316
0.0304
-0.0027999999999999982
0.02
-0.0092
0.002000000000000001
-0.006799999999999997
0.006400000000000001
0.0063999999

KeyboardInterrupt: ignored

In [ ]:
sns.heatmap(heatmap.transpose(), square = True, vmin = np.amin(heatmap), vmax = np.amax(heatmap), annot = False, fmt = ".4f", cmap = "YlGnBu")
plt.xticks([-1,5])
plt.yticks([0,0.5])
plt.show()

In [ ]:
"""avg_coops = np.array([0. for _ in range(50)])
for i in range(50):
    for trial in coops_lvl:
        avg_coops[i] += trial[i]
avg_coops = avg_coops/100"""

In [ ]:
def plot_coops(coops_lvl, r):
    x = np.arange(r)
    plt.plot(x, coops_lvl)
    plt.show()

In [ ]:
plot_coops(avg_coops, 50)

In [ ]:
lbls = np.random.choice(100, replace=False, size=100)
mapping = {i : lbls[i] for i in range(100)}
H = nx.relabel_nodes(G, mapping)